# CNN

## The project process

### 1. Prepare the data
### 2. Build the model
### 3. Train the model
### 4. Analyse the model's result

#### For preparation of the data, we will use the ETL process: Extract, Transform and Load.

In [2]:
import torch
import torchvision #library to access popular datasets, models
import torchvision.transforms as transforms

#### Now Fashion MNIST Data set

In [ ]:
train_set = torchvision.datasets.FashionMNIST(
            root='./data'
            )